In [3]:
#!pip install pairformance
# 

In [4]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

from examples.few_shot.scripts.collect_results import load_results, get_predictions_gold_probability_for_results_file

# prediction_gold_scores_hard: List[List[float]] = get_predictions_gold_probability_for_results_file(results_file, soft_scores=False, ensemble=True)

In [28]:
import os
import pprint
import pandas as pd
from pandas import DataFrame
import sys

import pairformance
from pairformance import plotting_utils

pp = pprint.PrettyPrinter(indent=4)

import numpy as np
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Filter the rows that we want to evaluate
def filter_by(df, constraints, skip_none_filter_vals=True):
    """Filter MultiIndex by sublevels.
       source: https://stackoverflow.com/questions/25224545/filtering-multiple-items-in-a-multi-index-python-panda-dataframe
    """
    constraints = {k: v if isinstance(v, list) else [v] for k,v in constraints.items() if v is not None}
    if len(constraints) == 0:
        return df
    
    indexer = [constraints[name] if name in constraints else slice(None)
               for name in df.index.names]
    return df.loc[tuple(indexer)] if len(df.shape) == 1 else df.loc[tuple(indexer),]

pd.Series.filter_by = filter_by
pd.DataFrame.filter_by = filter_by

# Loading our data 

In [29]:
# Load our results

results_raw_jsonl_list = ["/checkpoint/victorialin/few_shot/dense_7.5B_lang30_new_cc100_xl_unigram_mutli_tasks_v1/results.tsv.raw.jsonl",
                          "/checkpoint/victorialin/few_shot/dense_7.5B_lang30_new_cc100_xl_unigram_en_tasks/results.tsv.raw.jsonl"]

results_raw_jsonl_list = ["/large_experiments/xlmg/results/multilingual/dense_7_5B_30lang_cpts/results.tsv.raw.jsonl"]
all_results = []
for results_raw_file in results_raw_jsonl_list:
    results, expanded_results, current_tasks_to_groups  = load_results(results_raw_file)
    all_results.extend(results)

results = all_results
print(f"{len(results)} loaded")

917 loaded


# Process results
Filter some columns for faster processing

In [30]:
# Group by the unique keys

df = pd.DataFrame.from_records(results)

# these are the fields that we want to group by
index_cols = ["task", 
              "language",
              "eval_examples_cnt",
              "eval_set", 
              "train_set", 
              "train_lang",
              "nb_few_shot_samples", 
              "calibration",
              "template",
              "model_name", # we need the results file to load the predictions
             ]
value_cols_all = ["_metric_val",
                  "_metric_val_std",
              "results_file"]  # this is only the preferred value
value_cols = [x for x in value_cols_all if x in df.columns]
cols = []

df_selected = df[index_cols + value_cols + cols]

# debug - group by
# pt = pd.pivot_table(df_selected, values=value_cols, index=index_cols,
#                    columns=cols, aggfunc=[np.mean])
# pt = pt.swaplevel(1, 2, axis=1).sort_index(axis=1)
# pt

df_selected = df_selected.set_index(index_cols)  # this is for easy filtering
df_selected


_metric_val  \
task       language eval_examples_cnt eval_set train_set train_lang nb_few_shot_samples calibration template   model_name                                                        
storycloze en       1871              val2016  val2016   en         0                   False       storycloze dense_7.5B_lang30_new_cc100_xl_unigram__step000...    58.792090   
           ru       1871              val2016  val2016   ru         0                   False       storycloze dense_7.5B_lang30_new_cc100_xl_unigram__step000...    56.012827   
           zh       1871              val2016  val2016   zh         0                   False       storycloze dense_7.5B_lang30_new_cc100_xl_unigram__step000...    53.981828   
           ar       1871              val2016  val2016   ar         0                   False       storycloze dense_7.5B_lang30_new_cc100_xl_unigram__step000...    49.652592   
           sw       1871              val2016  val2016   sw         0                   False       storycloze dense_7.5B_lang30_new_cc100_xl_unigram__step000...    56.440406   
...                                                                                                                                                                        ...   
xwinograd  fr       83                test     test      fr         0                   False       winograd   dense_7.5B_lang30_new_cc100_xl_unigram__step000...    60.240964   
           jp       959               test     test      jp         0                   False       winograd   dense_7.5B_lang30_new_cc100_xl_unigram__step000...    54.848801   
           pt       263               test     test      pt         0                   False       winograd   dense_7.5B_lang30_new_cc100_xl_unigram__step000...    57.414449   
           ru       315               test     test      ru         0                   False       winograd   dense_7.5B_lang30_new_cc100_xl_unigram__step000...    56.825397   
           zh       16                test     test      zh         0                   False       winograd   dense_7.5B_lang30_new_cc100_xl_unigram__step000...    50.000000   

                                                                                                                                                                   _metric_val_std  \
task       language eval_examples_cnt eval_set train_set train_lang nb_few_shot_samples calibration template   model_name                                                            
storycloze en       1871              val2016  val2016   en         0                   False       storycloze dense_7.5B_lang30_new_cc100_xl_unigram__step000...              0.0   
           ru       1871              val2016  val2016   ru         0                   False       storycloze dense_7.5B_lang30_new_cc100_xl_unigram__step000...              0.0   
           zh       1871              val2016  val2016   zh         0                   False       storycloze dense_7.5B_lang30_new_cc100_xl_unigram__step000...              0.0   
           ar       1871              val2016  val2016   ar         0                   False       storycloze dense_7.5B_lang30_new_cc100_xl_unigram__step000...              0.0   
           sw       1871              val2016  val2016   sw         0                   False       storycloze dense_7.5B_lang30_new_cc100_xl_unigram__step000...              0.0   
...                                                                                                                                                                            ...   
xwinograd  fr       83                test     test      fr         0                   False       winograd   dense_7.5B_lang30_new_cc100_xl_unigram__step000...              0.0   
           jp       959               test     test      jp         0                   False       winograd   dense_7.5B_lang30_new_cc100_xl_unigram__step000...              0.0   
           pt       263     

# Filter data 
Filter here to improve the performance.
We will do pairwise comparison!  

In [33]:

filter_json = {
    'task': ["storycloze"], 
    'language': ["en"], 
    'eval_examples_cnt': None, 
    'eval_set': None, 
    'train_set': None,
    'train_lang': None, 
    'nb_few_shot_samples': None, 
    'calibration': None, 
    'template': None,
    'model_name': None
}



df_filtered = df_selected.filter_by(
    filter_json
)
# df_filtered.to_csv("results_debug.tsv")
df_filtered

_metric_val  \
task       language eval_examples_cnt eval_set train_set train_lang nb_few_shot_samples calibration template   model_name                                                        
storycloze en       1871              val2016  val2016   en         0                   False       storycloze dense_7.5B_lang30_new_cc100_xl_unigram__step000...    58.792090   
                                                                                                               dense_7.5B_lang30_new_cc100_xl_unigram__step000...    66.541956   
                                                                                                               dense_7.5B_lang30_new_cc100_xl_unigram__step000...    69.695350   
                                                                                                               dense_7.5B_lang30_new_cc100_xl_unigram__step000...    71.298771   
                                                                                                               dense_7.5B_lang30_new_cc100_xl_unigram__step000...    71.619455   
                                                                                                               dense_7.5B_lang30_new_cc100_xl_unigram__step000...    72.528060   
                                                                                                               dense_7.5B_lang30_new_cc100_xl_unigram__step000...    72.474613   

                                                                                                                                                                   _metric_val_std  \
task       language eval_examples_cnt eval_set train_set train_lang nb_few_shot_samples calibration template   model_name                                                            
storycloze en       1871              val2016  val2016   en         0                   False       storycloze dense_7.5B_lang30_new_cc100_xl_unigram__step000...              0.0   
                                                                                                               dense_7.5B_lang30_new_cc100_xl_unigram__step000...              0.0   
                                                                                                               dense_7.5B_lang30_new_cc100_xl_unigram__step000...              0.0   
                                                                                                               dense_7.5B_lang30_new_cc100_xl_unigram__step000...              0.0   
                                                                                                               dense_7.5B_lang30_new_cc100_xl_unigram__step000...              0.0   
                                                                                                               dense_7.5B_lang30_new_cc100_xl_unigram__step000...              0.0   
                                                                                                               dense_7.5B_lang30_new_cc100_xl_unigram__step000...              0.0   

                                                                                                                                                                                                        results_file  
task       language eval_examples_cnt eval_set train_set train_lang nb_few_shot_samples calibration template   model_name                                                                                             
storycloze en       1871              val2016  val2016   en         0                   False       storycloze dense_7.5B_lang30_new_cc100_xl_unigram__step000...  /large_experiments/xlmg/results/multilingual/d...  
                                                                                                               dense_7.5B_lang30_new_cc100_xl_unigram__step000...  /large_experiments/xlmg/results/multilingual/d...  
                                                             

### Load example data

In [34]:
predictions_by_index = {}

for index, row in df_filtered.iterrows():
    if index in predictions_by_index:
        # only unique values
        continue
    results_file = row["results_file"]
    if not os.path.exists(results_file):
        print("Does not exist:" + row["results_file"])
        continue
    
    prediction_gold_scores_correct = get_predictions_gold_probability_for_results_file(results_file, 
                                                                               soft_scores=False, # False == IsCorrect
                                                                               ensemble="mean") # this will ensemble
    predictions_by_index[index] = prediction_gold_scores_correct



In [26]:
eval_df = pd.read_csv('dialogue_example_df.csv')
eval_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'dialogue_example_df.csv'

In [ ]:
print(f" Systems: {eval_df.columns.to_list()},\n number of test instances: {eval_df.shape[0]}")

## Start Pairformance

In [ ]:
print("Default config:")
pp.pprint(pairformance.DEFAULT_CONFIG)

- `aggregations`: a list of aggregation mechanisms to consider
- `compute_pairwise`: whether to run the analyses for each pair of systems
- `epsilon_bt`: when the iterative algorithm of BT does not vary by more than epsilon_bt, we stop.
- `n_bootstrap`: number of bootstrap resample of the data to take to compute confidence intervals
- `pval_threshold`: percentile to use for confidence interval. Example: 0.05 gives the (5, 95) percentiles.

In [ ]:
pf_config = pairformance.DEFAULT_CONFIG
pf_config['n_bootstrap'] = 10

pf_eval = pairformance.Pairformance(df=eval_df, config=pf_config)

## Default Usage

In [ ]:
results = pf_eval.eval()
global_results = pf_eval.print_global_results()
pairwise_results = pf_eval.print_pairwise_results()

### Description of the returned variables

**Outputs from pf_eval.eval()**:
- `results['global-results']`: contains the global estimates for each aggregation mechanism defined in the config
- `results['global-results'][Agg]`: is a pd.DataFrame with n_bootstrap row, one row for one resampled estimate
- `results['pairwise-results']`: contains the pairwise comparison for each pair of systems

**Outputs from self.print_global_results()**: the same information as `results['global-results']` but with the confidence intervals computed from the resampled estimates.

**Outputs from self.print_pairwise_results()**: the same information as `results['pairwise-results']` but with the confidence intervals computed from the resampled estimates

## Probability that one system is better than other

In [ ]:
p_m0_m1, pval_m0_m1 = pf_eval.prob_a_better_b('M0', 'M1')
print(f"Prob that M0 is better than M1 {p_m0_m1}, pval {pval_m0_m1}")

## Visualization of results

In [ ]:
## Plot the distribution of BT estimates coming from bootstrap resamples
plotting_utils.plot_global_results(pf_eval, aggregation='BT')

In [ ]:
## Plot the distribution of mean estimates coming from bootstrap resamples
plotting_utils.plot_global_results(pf_eval, aggregation='Mean')

In [ ]:
## Plot pairwise structure for BT (BT pairwise is different than BT global)
plotting_utils.plot_matrix_results(pf_eval, aggregation='BT', fontsize=13, save_path=None)

In this matrix, the element at row i and column j is the probability that the system of row i is better than the system of column j

The star marks statistically significant difference according to the sign test (for BT)

In [ ]:
plotting_utils.plot_matrix_results(pf_eval, aggregation='Median', fontsize=13, save_path=None)

In this matrix, the element at row i and column j is the difference between the median of system of row i and the system of column j
The star marks statistically significant difference according to the Mood\'s median test (for Median)

## Diagnostic capabilities

### Computing statistics of the system scores

- `global-outliers-percentage`: take the whole array of data and compute the percentage of outliers
- `per-system-outliers-percentage`: percentage of outliers for each system independently
- `mean-row-std`: measure the std per test instance, then average this std across test instances, i.e., how much variance is there across system 
- `rank-std-across-rows`: computes the order of systems per test instance, and measure how much the ranking varies across test instances, i.e., how much variance is there across ranking of systems
- `std-of-row-means`: computes the mean score of each test instance (across systems) and measure the std across test instances, i.e., how much overall scores vary across test instances

In [ ]:
pp.pprint(pf_eval.diagnostic_system_scores(eval_df))

### Computing disagreements between the aggregation mechanisms for this data

In this case, we observe that there is good agreement between mean, median, and BT

In [ ]:
_ = pf_eval.diagnostic_aggregation_agreement()

### plotting paired graph

It show the paired structure of a pair of system. 
Each dot is one test instances whose coordinates are given by the score it receives according to the two systems.
This is shown as a density plot, if more density is above the dotted line the y-axis system is better according to BT.
Otherwise, the x-axis system is better according to BT.
See paper for more details!

In this case, the medians intersects below the dotted line, i.e., Median makes M1 better
More mass is on the space above the dotted line, i.e., BT makes M2 better
The means intersect on the dotted line, i.e., Mean does not distinguish between M2 and M1

In [ ]:
plotting_utils.plot_paired_graphs(eval_df, 'M2', 'M1')